In [1]:
import pandas as pd
import numpy as np

class FeatureFileGenerator(object):
    def __init__(self):
        np.random.seed(0) # seed so we always get same answer
    
    def new_file(self, filename):
        t_df = pd.read_csv('data/TourneyCompactResults.csv')
        t_df = self.randomize_teams(t_df)
        t_df['TourneyGame'] = np.full(t_df.shape[0], 1)
                
        #s_df = pd.read_csv('data/RegularSeasonCompactResults.csv')
        #s_df = self.randomize_teams(s_df)
        #s_df['TourneyGame'] = np.full(s_df.shape[0], 0)
        
        #df = pd.concat([t_df, s_df], ignore_index=True)
        t_df.to_csv(filename, encoding='ascii')
        
    def randomize_teams(self, df):
        df = df[df['Season'] > 2002]
        
        Ngames = df.shape[0]
        df['team0Win'] = np.random.randint(2,size=Ngames)
        
        for feature in [self.team0, self.team1]:
            df[feature.__name__] = df.apply(feature, axis=1)
        
        return df

    def team0(self, game):
        if game['team0Win'] == 1:
            return game['Wteam']
        else:
            return game['Lteam']

    def team1(self, game):
        if game['team0Win'] == 1:
            return game['Lteam']
        else:
            return game['Wteam']

In [6]:
import ipyparallel as ipp
c = ipp.Client()
view = c.load_balanced_view()

/Users/dtamayo/miniconda2/envs/p3/lib/python3.5/site-packages/ipyparallel/client/client.py:436: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@10.8.145.14')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


In [2]:
#%%px --local

import numpy as np
import pandas as pd

class FeatureAdder(object):
    def __init__(self, df):
        self.df = df
        
    def feature01(self, feature):
        def mod(game):
            f0 = team_features_df.loc[(team_features_df['Team_Id'] == game['team0']) & in_season[game['Season']], feature.__name__].values[0]
            f1 = team_features_df.loc[(team_features_df['Team_Id'] == game['team1']) & in_season[game['Season']], feature.__name__].values[0]
            return pd.Series({feature.__name__+'0':f0, feature.__name__+'1':f1})
        return mod
     
    def massey(self):
        self.massey_init()
        self.df = pd.concat([self.df, self.df.apply(self.massey_game, axis=1)], axis=1)

    def massey_init(self):
        self.massey_df = pd.read_csv('data/massey_ordinals_2003-2015.csv')
        
        # Create dictionaries for boolean masks

        self.massey_seasons = self.massey_df['season'].unique()
        self.in_massey_season = dict(zip(self.massey_seasons, [self.massey_df['season'] == i for i in self.massey_seasons]))

        self.massey_day_nums = self.massey_df['rating_day_num'].unique()
        self.is_massey_day_num = dict(zip(self.massey_day_nums, [self.massey_df['rating_day_num'] == i for i in self.massey_day_nums]))

        self.massey_teams = self.massey_df['team'].unique()
        self.is_massey_team = dict(zip(self.massey_teams, [self.massey_df['team'] == i for i in self.massey_teams]))

        self.massey_systems = self.massey_df['sys_name'].unique()
        self.is_massey_system = dict(zip(self.massey_systems, [self.massey_df['sys_name'] == i for i in self.massey_systems]))

    def massey_game(self, game): # add here any features specific to a game, i.e., how do teams interact       
        try:
            daynum = game['Daynum']
        except:
            daynum = 134 # submission file games don't have daynum, but are all after day 133 (last day before tourney)

        latest_day = np.max(self.massey_day_nums[self.massey_day_nums < daynum]) # only take ordinals from dates that happened before the game, and take latest one of those
        ordinals0 = [self.getOrdinal(game['Season'], self.massey_systems[i], latest_day, game['team0']) for i in range(len(self.massey_systems))]
        ordinals1 = [self.getOrdinal(game['Season'], self.massey_systems[i], latest_day, game['team1']) for i in range(len(self.massey_systems))]
        f0 = pd.Series(ordinals0, index=[self.massey_systems[i]+'0' for i in range(len(self.massey_systems))])   
        f1 = pd.Series(ordinals1, index=[self.massey_systems[i]+'1' for i in range(len(self.massey_systems))])   
        return pd.concat([f0,f1])
    
    def getOrdinal(self, season, system, day, team):
        try:
            ordinal = np.float64(self.massey_df.loc[self.in_massey_season[season] & self.is_massey_system[system] & self.is_massey_day_num[day] & self.is_massey_team[team], 'orank'].values[0])
        except IndexError:
            ordinal = np.nan
        return ordinal

def add_features(df):
    dfa = FeatureAdder(df)
    dfa.massey()
    return dfa.df

In [8]:
f = FeatureFileGenerator()
f.new_file('data/tourneydata.csv')

/Users/dtamayo/miniconda2/envs/p3/lib/python3.5/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/dtamayo/miniconda2/envs/p3/lib/python3.5/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/dtamayo/miniconda2/envs/p3/lib/python3.5/site-packages/numpy/core/numeric.py:294: FutureWarning: in the future, full(847, 1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


In [9]:
df = pd.read_csv('data/tourneydata.csv')

In [10]:
game = df.iloc[0]

In [13]:
df.tail()

,Unnamed: 0,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,team0Win,team0,team1,TourneyGame
842,1978,2015,146,1181,66,1211,52,N,0,1,1181,1211,1.0
843,1979,2015,146,1277,76,1257,70,N,1,1,1277,1257,1.0
844,1980,2015,152,1181,81,1277,61,N,0,1,1181,1277,1.0
845,1981,2015,152,1458,71,1246,64,N,0,0,1246,1458,1.0
846,1982,2015,154,1181,68,1458,63,N,0,0,1458,1181,1.0


In [11]:
print(game)

Unnamed: 0     1136
Season         2003
Daynum          134
Wteam          1421
Wscore           92
Lteam          1411
Lscore           84
Wloc              N
Numot             1
team0Win          0
team0          1411
team1          1421
TourneyGame       1
Name: 0, dtype: object


In [8]:
game['index']

KeyError: 'index'

In [23]:
Ncores=2*8
dfsplit = np.array_split(df,Ncores)
        
import os
view.map_sync(os.chdir, [os.getcwd()]*Ncores)
c[:].apply_sync(os.getcwd)

['/Users/dtamayo/Documents/NCAAbasketball',
 '/Users/dtamayo/Documents/NCAAbasketball']

In [24]:
dfs = view.map_sync(add_features, dfsplit)

In [25]:
dfnew = pd.concat([df for df in dfs])
dfnew.tail()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,team0Win,team0,...,STF1,WMR1,PPR1,MPI1,STS1,UPS1,SPR1,MvG1,TRK1,BWE1
1202,2004,136,1163,70,1436,53,N,0,0,1436,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1203,2004,136,1177,76,1173,69,N,2,0,1173,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1204,2004,136,1181,96,1106,61,N,0,0,1106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1205,2004,136,1211,76,1434,49,N,0,0,1434,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1206,2004,136,1264,75,1196,60,N,0,1,1264,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
dfnew.shape

(71, 276)

In [101]:
dfnew = pd.concat([df for df in dfs])
dfnew.to_csv('data/tourneydata.csv', encoding='ascii')